## This script runs a parameter distribution for WT and Bcl2 (x1.5) for the apoptosis model and plots the output (for figure 1E)

In [1]:
using Plots 
using CSV
using Distributions
using Random
using DataFrames
using JLD2
using DelimitedFiles
using FileIO
using Sundials
using LinearAlgebra
using DifferentialEquations

In [7]:
#set up where CSV2Julia is
locationOfCSV2Julia="csv2model-multiscale.py"

#identify the three CSV sheets that describe the model
reactionsFile="reactions.csv"
parametersFile="parameters.csv"
rateLawsFile="rateLaws_apop.csv"

#build a command to execute csv2julia
arguments=[reactionsFile, parametersFile, rateLawsFile, "odeApoptosis.jl"]
cmd=`python3 $locationOfCSV2Julia $arguments`

#lets run csv2julia (requires python to be installed)
run(cmd)

inline
Running CSV2JuliaDiffEq with parameters hard-coded into the CSV file, if this is not correct, re-run with the 5th argument set to 'scan' or 'param'
Opening rateLaws_apop.csv as rate law file
Opening parameters.csv as parameters file
Opening reactions.csv as reactions file


Process(`python3 csv2model-multiscale.py reactions.csv parameters.csv rateLaws_apop.csv odeApoptosis.jl`, ProcessExited(0))

In [8]:
#CV =  coefficient of variation
preConcCV=0.32 # how much should we distribute starting parameters for each cell
# We need a concentration CV of about 32% (if we were doing parameters it would be 11%ish)
# I've tried 32% and it seems too high, the effect of variability ends up way greater than the effect of
# the BCL-2 inhibition so I reduced this to 11%... can we justify that cell-to-cell variability is lower
# between cell lines than between primary cells? Probably.
# as reported here: https://www.pnas.org/content/115/12/E2888

#DISincrease=0.000001

0.32

In [9]:
function myLogNormal(m,std)
    γ = 1+std^2/m^2
    μ = log(m/sqrt(γ))
    σ = sqrt(log(γ))
    return LogNormal(μ,σ)
end

myLogNormal (generic function with 1 method)

In [10]:
#this function takes some initial conditions and distributes them by a set CV, returning new initial conditions
function sampleInit(y0,cv)
    #note mean of distribution is 1 so CV = standard deviation
    distribution=Truncated(myLogNormal(1,cv),0,Inf)
    scaling=rand(distribution, 1)
    scalingMatrix=[ rand(distribution, 1)[1] for i=1:length(y0)]
    ynew=y0.*scalingMatrix
    return ynew
end

sampleInit (generic function with 1 method)

In [11]:
odeFile=include("odeApoptosis.jl")
include("variableNames.jl")
p=1
maxTimeSS=100000.0
maxTimeTC=144*60.0
params=1

1

In [12]:
# fix species
include("fixSpecies.jl")
fixSpecies("odeApoptosis.jl","odeApoptosisFixed.jl",1)
odeFile = include("odeApoptosisFixed.jl")

odeApoptosis! (generic function with 1 method)

In [13]:
## Bcl2t values:

# WT  = 277
# 1.5 = 415.5
# 2.0 = 554
# 2.5 = 692.5
# 3.0 = 831
# 3.5 = 969.5

# replace xxx with value for copy number

function initConditionsApop(y0,syms)
   #units: M
   #source: the supplement of Albeck et al 2008 PLOS Biology
   y0[findfirst(isequal("L"),syms)]=0
   y0[findfirst(isequal("R"),syms)]=1000
   y0[findfirst(isequal("flip"),syms)]=2000
   y0[findfirst(isequal("pC8"),syms)]=10000
   y0[findfirst(isequal("BAR"),syms)]=1000
   y0[findfirst(isequal("pC3"),syms)]=10000
   y0[findfirst(isequal("pC6"),syms)]=10000
   y0[findfirst(isequal("XIAP"),syms)]=100000
   y0[findfirst(isequal("PARP"),syms)]=1000000
   y0[findfirst(isequal("Bid"),syms)]=60000
   y0[findfirst(isequal("Bcl2c"),syms)]=20000
   y0[findfirst(isequal("Bax"),syms)]=80000
   y0[findfirst(isequal("Bcl2"),syms)]=30000
   y0[findfirst(isequal("Mito"),syms)]=500000
   y0[findfirst(isequal("mCytoC"),syms)]=500000
   y0[findfirst(isequal("mSMac"),syms)]=100000
   y0[findfirst(isequal("Apaf"),syms)]=100000
   y0[findfirst(isequal("Procasp9"),syms)]=100000
   #this is new and will be replaced by cRel control in multiscale modeling
   y0[findfirst(isequal("Bcl2t"),syms)]=277
    return y0
end

initConditionsApop (generic function with 1 method)

In [14]:
global f=ODEFunction(odeFile,syms=Symbol.(syms))

(::ODEFunction{true, SciMLBase.FullSpecialize, typeof(odeApoptosis!), UniformScaling{Bool}, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Nothing, Vector{Symbol}, Nothing, Nothing, typeof(SciMLBase.DEFAULT_OBSERVED), Nothing, Nothing}) (generic function with 1 method)

In [15]:
import Base.Threads

In [16]:
conditions=["WT","Bcl2_mut"]
bcl2Vals=[1,1.5]

2-element Vector{Float64}:
 1.0
 1.5

In [17]:
first_cell = 1
last_cell = 1000

1000

### Steady state

In [18]:
function genSSconds(Bcl2,mut)
    y0=zeros(length(syms))
    y0=initConditionsApop(y0,syms)
    y0[findfirst(isequal("L"),syms)]=0
    y0[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

    #solve the steady state
    prob=ODEProblem(f,y0,(0.0,maxTimeSS),abstol=1e-5,reltol=1e-3, maxiters=1e10)
    solss=solve(prob, Rosenbrock23(), saveat=maxTimeSS/1000)

    #dynamic phase, use SS solution as initial conditions
    y0_ss=vec(convert(Array, solss[:,end]))    
    y0_ss[findfirst(isequal("L"),syms)]=1

    # save y0 as csv
    writedlm("steady_state_"*mut*".csv", y0_ss, ',')
    println("steady state conditions saved")
end

genSSconds (generic function with 1 method)

In [19]:
genSSconds(1,"WT")

steady state conditions saved


In [25]:
genSSconds(1.5,"Bcl2_mut")

steady state conditions saved


In [21]:
function genRunConds(Bcl2, condition_folder)
    Random.seed!(1234) #for reproducibility
    for i in first_cell:last_cell
        #import SS conds
        y0_ss = readdlm("steady_state_WT.csv")
        #set L value
        y0_ss[findfirst(isequal("L"),syms)]=1
        
        #distribute all initial conditions
        y0_ss = sampleInit(y0_ss,preConcCV)

        # set Bcl2 value
        y0_ss[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

        # save y0 as csv
        writedlm("SS_conds/"*condition_folder*"/SS_conds_"*string(i)*".csv", y0_ss, ',')    
        println("Run conditions saved")
    end
end

genRunConds (generic function with 1 method)

In [22]:
function runSimSS(first_cell, last_cell, Bcl2, thisCondition)
    Threads.@threads for i in first_cell:last_cell
        solveSS(i, Bcl2, thisCondition)
    end
end

runSimSS (generic function with 1 method)

### Run full sim

In [23]:
function genInitConds(first_cell, last_cell, Bcl2, thisCondition)
    #import SS conds
    y0_ss = vec(readdlm("steady_state_"*thisCondition*".csv"))
    
    for i in first_cell:last_cell
        #distribute all initial conditions
        y0_init = sampleInit(y0_ss,preConcCV)
                
        # set Bcl2 value
        y0_init[findfirst(isequal("Bcl2t"),syms)]=277*Bcl2

        # save y0 as csv
        writedlm("SS_conds/"*thisCondition*"/SS_conds_"*string(i)*".csv", y0_init, ',')
    end
    println("conditions for "*thisCondition*" saved")
end

genInitConds (generic function with 1 method)

In [26]:
for condIndex in 1:length(conditions)
    #set seed
    Random.seed!(1234) #for reproducibility
    #get conditions
    thisCondition=conditions[condIndex]
    thisBCL2Scale=bcl2Vals[condIndex]
    #create folder to save SS conditions
    mkpath("SS_conds/"*thisCondition*"/")
    #run simulations
    genInitConds(first_cell, last_cell, thisBCL2Scale, thisCondition)
end

conditions for WT saved
conditions for Bcl2_mut saved


In [27]:
#find cleaved parp
speciesToPlot="CPARP"
#find total PARP
denominator=["PARP","PARP_C3","CPARP"]
CPARPIndex=findfirst(isequal(speciesToPlot),syms)
denominatorIndexes=Array{Int64}(undef,length(denominator))
for i in 1:length(denominator)
    thisIndex=findfirst(isequal(denominator[i]),syms)
    denominatorIndexes[i]=convert(Int64,thisIndex)
end

In [33]:
function calcT2D(sol, i, thisCondition)
    cParray=zeros(1,Int(maxTimeTC)+1)
    thisNumerator=sol[CPARPIndex,:] #cPARP
    thisDenominatorTCs=sol[denominatorIndexes,:]
    thisDenominator=sum(thisDenominatorTCs,dims=1) #other PARP species
    parpPercentage=thisNumerator./thisDenominator'
    cParray=[cParray;parpPercentage']
    if cParray[end] > 0.01
        T2Death = findfirst(x->x>0.01, cParray)            
        T2D_t = T2Death[2]
        T2D_df = DataFrame(Cell="cell_"*string(i), T2D=T2D_t)
        CSV.write("T2D_res/T2D_res_"*thisCondition*"/cell_"*string(i)*"_res.csv", T2D_df)
    else
    end
end

calcT2D (generic function with 2 methods)

In [34]:
function solveCell(i, Bcl2, thisCondition)    
        
    for i in first_cell:last_cell
        
        y0_init = vec(readdlm("SS_conds/"*thisCondition*"/SS_conds_"*string(i)*".csv"))
        
        prob=ODEProblem(f,y0_init,(0.0,maxTimeTC),abstol=1e-5,reltol=1e-3, maxiters=1e10)
        sol=solve(prob, Rosenbrock23(), saveat=1.0, progress = true)

        calcT2D(sol, i, thisCondition)

    end
    println(thisCondition*" finished")
end

solveCell (generic function with 1 method)

In [30]:
function runSim(first_cell, last_cell, Bcl2, thisCondition)
    Threads.@threads for i in first_cell:last_cell
            solveCell(i, Bcl2, thisCondition)
    end
end

runSim (generic function with 1 method)

### Run simulation

In [ ]:
for condIndex in 1:length(conditions)
    #set seed
    Random.seed!(1234) #for reproducibility
    #get conditions
    thisCondition=conditions[condIndex]
    thisBCL2Scale=bcl2Vals[condIndex]
    #create folder to save SS conditions
    mkpath("T2D_res/T2D_res_"*thisCondition*"/")
    #run simulations
    runSim(first_cell, last_cell, thisBCL2Scale, thisCondition)
end

In [ ]:
#set seed
Random.seed!(1234) #for reproducibility
#get conditions
thisCondition=conditions[2]
thisBCL2Scale=bcl2Vals[2]
#create folder to save SS conditions
#mkpath("T2D_res/T2D_res_"*thisCondition*"/")
#run simulations
runSim(first_cell, last_cell, thisBCL2Scale, thisCondition)

### Code for importing results and plotting

In [ ]:
### Get T2D times

function getT2Dtimes(mut_folder)
    T2D_times_df = DataFrame(Cell=[], T2D=Int64[])

    for i in first_cell:last_cell
        if isfile("T2D_res/T2D_res_"*mut_folder*"/cell_"*string(i)*"_res.csv") == true
            T2D_times = DataFrame(CSV.File("T2D_res/T2D_res_"*mut_folder*"/cell_"*string(i)*"_res.csv", header=true))
            append!(T2D_times_df, T2D_times)
        else
            continue
        end
    end
    return(T2D_times_df)
end

In [ ]:
WT_res = getT2Dtimes("WT")
rename!(WT_res,:T2D => :WT)

In [ ]:
Bcl2_mut_res = getT2Dtimes("Bcl2_mut")
rename!(Bcl2_mut_res,:T2D => :Bcl2_mut)

In [ ]:
WT_res_mod = filter(:Cell => in(Bcl2_mut_res.Cell), WT_res)

In [ ]:
WT_Bcl2_res = innerjoin(WT_res_mod, Bcl2_mut_res, on = :Cell)
WT_Bcl2_res = sort!(WT_Bcl2_res, [:WT], rev=true)

In [ ]:
bar(WT_Bcl2_res.Bcl2_mut, orientation=:h, size=(900,900), linewidth = 0, color=:green, alpha=0.5, legend=false)
bar!(WT_Bcl2_res.WT, orientation=:h, size=(900,900), linewidth = 0, color=:black, alpha=0.5, legend=false)
s=string.(collect(0:20:140))
plot!(xticks = (0:(60*20):(60*140),s),xlabel="time (hours)",ylabel="cell number",fmt = :png,
        xtickfontsize=22,
        ytickfontsize=22,
        xguidefontsize=22,
        yguidefontsize=22)